In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0-beta1
sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)
matplotlib 3.0.3
numpy 1.16.2
pandas 0.24.2
sklearn 0.20.3
tensorflow 2.0.0-beta1
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [2]:
loaded_keras_model = keras.models.load_model(
    './graph_def_and_weights/fashion_mnist_model.h5')
loaded_keras_model(np.ones((1, 28, 28)))

W0714 21:49:37.675129 140736297124800 deprecation.py:323] From /Users/zhangyx/workspace/environments/tf2_py3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


<tf.Tensor: id=497, shape=(1, 10), dtype=float32, numpy=
array([[0.20111723, 0.09845883, 0.02821723, 0.0199804 , 0.03568273,
        0.07881083, 0.10847212, 0.02893549, 0.34842488, 0.05190022]],
      dtype=float32)>

In [3]:
keras_to_tflite_converter = tf.lite.TFLiteConverter.from_keras_model(
    loaded_keras_model)
keras_to_tflite_converter.optimizations = [
    tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
keras_tflite = keras_to_tflite_converter.convert()

In [4]:
if not os.path.exists('./tflite_models'):
    os.mkdir('./tflite_models')
with open('./tflite_models/quantized_keras_tflite', 'wb') as f:
    f.write(keras_tflite)

In [5]:
run_model = tf.function(lambda x : loaded_keras_model(x))
keras_concrete_func = run_model.get_concrete_function(
    tf.TensorSpec(loaded_keras_model.inputs[0].shape,
                  loaded_keras_model.inputs[0].dtype))

In [6]:
keras_concrete_func(tf.constant(np.ones((1, 28, 28), dtype=np.float32)))

<tf.Tensor: id=650, shape=(1, 10), dtype=float32, numpy=
array([[0.20111723, 0.09845883, 0.02821723, 0.0199804 , 0.03568273,
        0.07881083, 0.10847212, 0.02893549, 0.34842488, 0.05190022]],
      dtype=float32)>

In [7]:
concrete_func_to_tflite_converter = tf.lite.TFLiteConverter.from_concrete_functions([keras_concrete_func])
concrete_func_to_tflite_converter.optimizations = [
    tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
concrete_func_tflite = concrete_func_to_tflite_converter.convert()

In [8]:
with open('./tflite_models/quantized_concrete_func_tflite', 'wb') as f:
    f.write(concrete_func_tflite)

In [9]:
saved_model_to_tflite_converter = tf.lite.TFLiteConverter.from_saved_model('./keras_saved_graph/')
saved_model_to_tflite_converter.optimizations = [
    tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
saved_model_tflite = saved_model_to_tflite_converter.convert()

In [10]:
with open('./tflite_models/quantized_saved_model_tflite', 'wb') as f:
    f.write(saved_model_tflite)